In [1]:
import numpy as np

In [2]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y - t)**2)

t = [0,0,1,0,0]
y = [0.1,0.05,0.6,0,0.25]

mean_squared_error(np.array(y), np.array(t))

0.11750000000000002

In [3]:
def cross_entropy_error(y, t):
    delta = 1e-7
    
    return -np.sum(t * np.log(y + delta))

In [4]:
t = [0,0,1.0,0,0]
y = [0.1, 0.2, 0.5, 0.05, 0.15]

cross_entropy_error(np.array(y), np.array(t))

0.6931469805599654

# mini-batch

In [5]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')

dat = mnist.data.to_numpy()
tar = mnist.target.to_numpy()

x_train = dat[:60000,]
x_test = dat[60000:,]
t_train = tar[:60000]
t_test = tar[60000:]

C:\Users\nemok\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [6]:
print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000,)


In [7]:
I = np.eye(10)
one_hot = I[t_train.astype(int)]

print(one_hot)

t_train = one_hot

[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [8]:
print(t_train.shape)

(60000, 10)


In [9]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)

x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [10]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [11]:
def cross_entropy_error_with_number_label(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    # y[np.arange(batch_size), t] = y[x(각 배치의 인덱스), t(해당 레이블의 답)] -> 신경망 output은 one-hot-encoding 상태
    return -np.sum(np.log(y[np.arange(batch_size, t) + 1e-7]) / batch_size)

# differential

In [12]:
def numerical_diff_1(f, x):
    h = 10e-50
    return (f(x + h) - f(x)) / h

In [13]:
# y = x^2
# y' = 2x
def func(x):
    return x**2

In [14]:
numerical_diff_1(func, 5)

0.0

In [15]:
def numerical_diff_2(f, x):
    h = 0.0001
    return (f(x+h) - f(x-h)) / (2*h)

In [16]:
numerical_diff_2(func, 5)

9.999999999976694

In [17]:
def partial_func(x):
    return np.sum(x**2)

In [18]:
def function_tmp1(x0):
    return x0*x0 + 4.0**2.0

numerical_diff_2(function_tmp1, 3.0)

6.00000000000378

In [19]:
def function_tmp2(x1):
    return 3.0**2.0 + x1*x1

numerical_diff_2(function_tmp2, 4.0)

7.999999999999119

In [20]:
def numerical_gradient(f, x):
    h = 0.00001
    grad = np.zeros_like(x)
    
    for i in range(x.size):
        tmp = x[i]
        
        x[i] = tmp + h
        f_up = f(x)
        
        x[i] = tmp - h
        f_down = f(x)
        
        x[i] = tmp
        grad[i] = (f_up - f_down) / (2 * h)
        
    return grad
        

In [21]:
print(numerical_gradient(partial_func, np.array([3.0, 4.0])))
print(numerical_gradient(partial_func, np.array([0.0, 2.0])))
print(numerical_gradient(partial_func, np.array([3.0, 0.0])))

[6. 8.]
[0. 4.]
[6. 0.]


In [22]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= (lr * grad)
        
    return x

In [23]:
def function_2(x):
    return x[0] ** 2 + x[1] ** 2

init_x = np.array([-2.7, 5.5])

print(gradient_descent(function_2, init_x))

[-0.3580728   0.72940756]


# hyper parameter test

In [24]:
print(gradient_descent(function_2, init_x, step_num=250))

[-0.00229346  0.00467185]


In [25]:
print(gradient_descent(function_2, init_x, lr=0.1))

[-4.67185070e-13  9.51673292e-13]


In [26]:
print(gradient_descent(function_2, init_x, lr=2))

[ 7.77852890e+10 -3.38901086e+11]


In [27]:
print(gradient_descent(function_2, init_x, lr=0.1, step_num=300))

[ 7.77852890e+10 -3.38901086e+11]


In [28]:
def softmax(x):
    C = np.max(x)
    e = np.exp(x - C)
    s = np.sum(e)
    
    return e / s

In [29]:
class SimpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)
        
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [30]:
net = SimpleNet()

print(net.W)

x = np.array([0.6, 0.9])
p = net.predict(x)

print(p)
print(np.argmax(p))

t = np.array([0, 0, 1])
net.loss(x, t)

[[ 0.86335408 -0.55433456 -0.23944542]
 [ 0.68025323  0.05048965 -1.47009898]]
[ 1.13024035 -0.28716005 -1.46675633]
0


2.8722299796497683

In [31]:
def numerical_gradient_2d(f, x):
    h = 0.00001
    grad = np.zeros_like(x)
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            tmp = x[i, j]

            x[i, j] = tmp + h
            f_up = f(x)

            x[i, j] = tmp - h
            f_down = f(x)

            x[i, j] = tmp
            grad[i, j] = (f_up - f_down) / (2 * h)
        
    return grad

In [32]:
def f(W):
    return net.loss(x, t)

In [33]:
dW = numerical_gradient_2d(f, net.W)

print(dW)

[[ 0.45563536  0.11042013 -0.56605548]
 [ 0.68345303  0.16563019 -0.84908322]]


In [34]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# make neural net

In [35]:
class two_layer_net:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}

        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient_2d(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient_2d(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

In [36]:
net = two_layer_net(input_size=784, hidden_size=100, output_size=10)

In [37]:
x = np.random.rand(100, 784)
y = net.predict(x)

In [38]:
x = np.random.rand(100, 784)
t = np.random.rand(100, 10)

grads = net.numerical_gradient(x, t)

In [39]:
print(grads['W1'].shape)
print(grads['b1'].shape)
print(grads['W2'].shape)
print(grads['b2'].shape)

(784, 100)
(100,)
(100, 10)
(10,)


# mini batch

In [40]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')

dat = mnist.data.to_numpy()
tar = mnist.target.to_numpy()

x_train = dat[:60000,]
x_test = dat[60000:,]
t_train = tar[:60000]
t_test = tar[60000:]

C:\Users\nemok\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [41]:
I = np.eye(10)
t_train = I[t_train.astype(int)]
t_test = I[t_test.astype(int)]

In [43]:
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = 50

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = two_layer_net(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.numerical_gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc : " + str(train_acc) + " | " + str(test_acc))

train acc, test acc : 0.1243 | 0.1231
train acc, test acc : 0.6831333333333334 | 0.6872


C:\Users\nemok\AppData\Local\Temp\ipykernel_14700\1253679003.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


train acc, test acc : 0.7819833333333334 | 0.7827


KeyboardInterrupt: 